In [0]:
!pip3 install torchgan

     |████████████████████████████████| 61kB 2.9MB/s 


In [ ]:
import cv2
import os
import torch
import random
import matplotlib.pyplot as plt
import numpy as np
# Pytorch and Torchvision Imports
import torch
import torchvision
from torch.optim import Adam
import torch.nn as nn
import torch.utils.data as data
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torchvision.utils as vutils
# Torchgan Imports
from torchgan.losses import MinimaxGeneratorLoss, MinimaxDiscriminatorLoss
import torchgan
from torchgan.layers import SpectralNorm2d, SelfAttention2d, ResidualBlock2d
from torchgan.models import Generator, Discriminator
from torchgan.losses import WassersteinGeneratorLoss, WassersteinDiscriminatorLoss, WassersteinGradientPenalty
from torchgan.trainer import Trainer
from google.colab.patches import cv2_imshow

In [0]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def mnist_dataloader():
    train_dataset = dsets.MNIST(root="./mnist", train=True, download=True,
                      transform=transforms.Compose([transforms.Resize((32, 32)),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize(mean=(0.5,),
                                                                         std=(0.5,))]))
    train_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=True)
    return train_loader


In [0]:
def apple_loader(data_size=512):
    for root, dirs, files in os.walk("../content/drive/My Drive/apples/images/"):
        res = np.zeros((data_size, 3,128,128))
        ex_count = 0
        for i, file in enumerate(files):
        
     
            if file.endswith(".jpg"):
                image=cv2.imread(root+file)
                try:
                    image.resize(3,128,128)
                    image = image/ 127.5 - 1.
                    res[i,:,:,:] = image
                except AttributeError:
                    ex_count+=1
                    pass
             
            if i > data_size-2 : 
                print(ex_count)
                print(len(files))
                features = torch.from_numpy(res)
                features = features.float()
                targets = torch.ones(data_size)
                train_tensor = data.TensorDataset(features, targets)
                train_loader = data.DataLoader(train_tensor, batch_size=data_size//4, shuffle=True)
                return train_loader

In [0]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    # Use deterministic cudnn algorithms
    torch.backends.cudnn.deterministic = True
    epochs = 20
else:
    device = torch.device("cpu")
    epochs = 10

print("Device: {}".format(device))
print("Epochs: {}".format(epochs))

Device: cuda:0
Epochs: 20


In [0]:
class ResUp(nn.Module):
    def __init__(self, step_channels=64):
        super(ResUp, self).__init__()
        d = step_channels
        self.skip = nn.Sequential(
                      nn.Upsample(),
                      nn.Conv2d(d * 2, d * 4, 1))
        self.model = nn.Sequential(
                    nn.BatchNorm2d(d), nn.LeakyReLU(0.2),
                    nn.Upsample(),
                    nn.Conv2d(d * 2, d * 4, 3),
                    nn.BatchNorm2d(d * 2), nn.LeakyReLU(0.2),
                    nn.Conv2d(d * 2, d * 4, 3))
    def forward(self, x):
        main = self.model(x)
        skip = self.skip(x)
        out = torch.add(main, skip)
        return out


In [0]:
class ResDown(nn.Module):
    def __init__(self, step_channels=64):
        super(ResDown, self).__init__()
        d = step_channels
        self.skip = nn.Sequential(
                      nn.Conv2d(d * 2, d * 4, 1),
                      nn.AvgPool2d(2))
        self.model = nn.Sequential(
                    nn.LeakyReLU(0.2),
                    nn.Conv2d(d * 2, d * 4, 3),
                    nn.LeakyReLU(0.2),
                    nn.Conv2d(d * 2, d * 4, 3),
                    nn.AvgPool2d(2))
    def forward(self, x):
        main = self.model(x)
        skip = self.skip(x)
        out = torch.add(skip, main)
        return out

In [0]:
class BigGanDiscriminator(Discriminator):
    def __init__(self, chn=64):
#         chn = step_channels
        super(BigGanDiscriminator, self).__init__(3, 'none')
        self.pre_conv = nn.Sequential(SpectralNorm2d(nn.Conv2d(3, 1*chn, 3)),
                                      nn.ReLU(),
                                      SpectralNorm2d(nn.Conv2d(1*chn, 1*chn, 3)),
                                      nn.AvgPool2d(2))
        self.pre_skip = SpectralNorm2d(nn.Conv2d(3, 1*chn, 1))

        self.conv = nn.Sequential(ResDown(1*chn),
                                  SelfAttention2d(64*64),
                                  ResDown(1*chn),    
                                  ResDown(2*chn),
                                  ResDown(4*chn),
                                  ResDown(8*chn))
        self.res_layer = ResidualBlock2d([16*chn, 16*chn], [3])
        self.linear = SpectralNorm2d(nn.Linear(4 * 4 * 16 * chn, 1))
        self.last = nn.LeakyReLU(0.2)
    def forward(self, x):
      out = self.pre_conv(x)
      out = self.pre_skip(out)
      out = self.conv(out)
      out = self.res_layer(out)
      out = self.linear(out)
      out = self.last(out)
      out = out.sum(2)
      return out

In [0]:
class BigGanGenerator(Generator):
    def __init__(self, chn=64):
        super(BigGanGenerator, self).__init__(100, 'none')
        self.linear = SpectralNorm2d(nn.Linear(100, 4 * 4 * 16 * chn))
        self.conv = nn.Sequential(ResUp(16*chn),
                                ResUp(16*chn),
                                ResUp(8*chn),
                                ResUp(4*chn),
                                SelfAttention2d(2*chn),
                                ResUp(2*chn))
        self.last = nn.Sequential(nn.BatchNorm2d(chn), nn.LeakyReLU(0.2),
                nn.Conv2d(chn * 2, 3, 3), nn.Tanh())
    def forward(self, x):
        out = self.linear(x)
        out = out.view(-1, 16*32, 4, 4)
        out = self.conv(out)
        out = self.last(out)
        return out

In [0]:
network_params = {
    "generator": {"name": BigGanGenerator, "args": {"chn": 32},
                  "optimizer": {"name": Adam, "args": {"lr": 0.0001, "betas": (0.0, 0.999)}}},
    "discriminator": {"name": BigGanDiscriminator, "args": {"chn": 32},
                      "optimizer": {"name": Adam, "args": {"lr": 0.0004, "betas": (0.0, 0.999)}}}
}

In [0]:
trainer = Trainer(network_params, [MinimaxGeneratorLoss(), MinimaxDiscriminatorLoss()], sample_size=64, epochs=epochs, device=device)

In [138]:
trainer(apple_loader(128))

0
1162


ValueError: ignored